In [7]:
from pyspark.sql import SparkSession
from pyspark.sql import Row
from delta import *

# warehouse_location points to the default location for managed databases and tables
warehouse_location = 'hdfs://hdfs-nn:9000/warehouse'

builder = SparkSession \
    .builder \
    .appName("Python Spark SQL Hive integration example") \
    .config("spark.sql.warehouse.dir", warehouse_location) \
    .config("hive.metastore.uris", "thrift://hive-metastore:9083") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.jars.packages", "io.delta:delta-core_2.12:1.0.0") \
    .enableHiveSupport() \

spark = spark = configure_spark_with_delta_pip(builder).getOrCreate()

In [15]:
reciclagem = spark.table("PL7G1_SILVER.Reciclagem")
incendios = spark.table("PL7G1_SILVER.Incendios")

In [16]:
### Get Month from date in pyspark
 
from pyspark.sql.functions import month,col,lit,when, col, concat, split
 
incendios = incendios.withColumn('month',month(incendios.date))

incendios = incendios.groupBy("borough","case_year","month").pivot("fire_code_category").count()
incendios = incendios.na.fill(value=0)
incendios = incendios.withColumn('Total_Fires', incendios.Appliances + col("Cooking Carelessness") + incendios.Electrical+col("Electrical Devices") + incendios.Heaters+ col("Hot Objects")+ incendios.Ignitable + incendios.Incendiary+ incendios.Motors + col("Natural Sources") + col("No Fire")+ col("Open Flame") + incendios.Other + col("Sem Informacao") + incendios.Smoking)
incendios = incendios.withColumn("month_name",lit(None)) 

incendios = incendios.withColumn("month_name", 
   when((incendios.month == "1"), lit("January"))
     .when((incendios.month == "2"), lit("February")) 
     .when((incendios.month == "3"), lit("March")) 
     .when((incendios.month == "4"), lit("April")) 
     .when((incendios.month == "5"), lit("May")) 
     .when((incendios.month == "6"), lit("June")) 
     .when((incendios.month == "7"), lit("July")) 
     .when((incendios.month == "8"), lit("August")) 
     .when((incendios.month == "9"), lit("September")) 
     .when((incendios.month == "10"), lit("October"))   
     .when((incendios.month == "11"), lit("November"))  
     .when((incendios.month == "12"), lit("December")))

#Tirar os " " e meter "_"
from pyspark.sql import functions as F
incendios = incendios.select([F.col(col).alias(col.replace(' ', '_')) for col in incendios.columns])


In [17]:
incendios.toPandas()

borough  case_year  month  Appliances  Cooking_Carelessness  \
0    Staten Island       2018      1           1                     0   
1    Staten Island       2017      6           4                     1   
2            Bronx       2019      2           2                     3   
3            Bronx       2019      3           3                     9   
4            Bronx       2016      7           1                     5   
..             ...        ...    ...         ...                   ...   
341  Staten Island       2017      1           2                     1   
342      Manhattan       2016      7           4                     1   
343         Queens       2020      3           1                     6   
344      Manhattan       2019     10           1                     5   
345      Manhattan       2016      1           3                     3   

     Electrical  Electrical_Devices  Heaters  Hot_Objects  Ignitable  \
0             4                   0        1            1          1   
1             1                   0        0            1          0   
2            16                   1        2            3          1   
3            14                   0        3            6          0   
4            11                   1        0            5          0   
..          ...                 ...      ...          ...        ...   
341           1                   0        2            0          1   
342          17                   0        0            7          0   
343          15                   0        3            1          2   
344          12                   1        1            6          2   
345          11                   1        3            2          0   

     Incendiary  Motors  Natural_Sources  No_Fire  Open_Flame  Other  \
0             7       1                0        2           1     12   
1             4       0                0        1           1     10   
2            26       2                0        6           7     29   
3            30       0                0        7           5     31   
4            29       2                2        7           5     56   
..          ...     ...              ...      ...         ...    ...   
341           9       0                0        0           3      7   
342          38       1                3        4           7     49   
343          18       1                1        3           4     13   
344          26       3                2        7           3     39   
345          38       1                0        8           9     43   

     Sem_Informacao  Smoking  Total_Fires month_name  
0                 0        3           34    January  
1                 0        2           25       June  
2                 0       16          114   February  
3                 0       20          128      March  
4                 0       18          142       July  
..              ...      ...          ...        ...  
341               0        5           31    January  
342               0       20          151       July  
343               0       12           80      March  
344               0        5          113    October  
345               0       24          146    January  

[346 rows x 20 columns]

In [21]:
from pyspark.sql.functions import substring, avg, sum, max, min
import pyspark.sql.functions as sf

# read data from the silver tables
reciclagem = spark.table("PL7G1_SILVER.Reciclagem")

reciclagem = reciclagem \
           .groupBy("Year", "Borough","Month_Name") \
           .agg(sum(reciclagem.Diversion_Rate_Total).alias("SUM_Diversion_Rate_Total"),
                avg(reciclagem.Diversion_Rate_Total).alias("AVG_Diversion_Rate_Total"),
                max(reciclagem.Diversion_Rate_Total).alias("MAX_Diversion_Rate_Total"),
                min(reciclagem.Diversion_Rate_Total).alias("MIN_Diversion_Rate_Total"),
                sum(reciclagem.Capture_Rate_Paper).alias("SUM_Capture_Rate_Paper"),
                avg(reciclagem.Capture_Rate_Paper).alias("AVG_Capture_Rate_Paper"),
                max(reciclagem.Capture_Rate_Paper).alias("MAX_Capture_Rate_Paper"),
                min(reciclagem.Capture_Rate_Paper).alias("MIN_Capture_Rate_Paper"),
                sum(reciclagem.Capture_Rate_MGP).alias("SUM_Capture_Rate_MGP"),
                avg(reciclagem.Capture_Rate_MGP).alias("AVG_Capture_Rate_MGP"),
                max(reciclagem.Capture_Rate_MGP).alias("MAX_Capture_Rate_MGP"),
                min(reciclagem.Capture_Rate_MGP).alias("MIN_Capture_Rate_MGP"),
                sum(reciclagem.Capture_Rate_Total).alias("SUM_Capture_Rate_Total"),
                avg(reciclagem.Capture_Rate_Total).alias("AVG_Capture_Rate_Total"),
                max(reciclagem.Capture_Rate_Total).alias("MAX_Capture_Rate_Total"),
                min(reciclagem.Capture_Rate_Total).alias("MIN_Capture_Rate_Total"),
               )

In [29]:
reciclagem.toPandas()

Year        Borough Month_Name  SUM_Diversion_Rate_Total  \
0    2017         Queens   November                     287.3   
1    2017  Staten Island   February                      61.2   
2    2017         Queens     August                     245.2   
3    2017      Manhattan    October                     245.5   
4    2017      Manhattan     August                     232.5   
..    ...            ...        ...                       ...   
235  2019         Queens       June                     269.3   
236  2019       Brooklyn    October                     296.7   
237  2019       Brooklyn   November                     315.8   
238  2019      Manhattan   December                     263.7   
239  2019  Staten Island  September                      60.6   

     AVG_Diversion_Rate_Total  MAX_Diversion_Rate_Total  \
0                   20.521429                      28.2   
1                   20.400000                      21.4   
2                   17.514286                      23.0   
3                   20.458333                      27.8   
4                   19.375000                      26.0   
..                        ...                       ...   
235                 19.235714                      23.8   
236                 16.483333                      29.0   
237                 17.544444                      29.8   
238                 21.975000                      29.5   
239                 20.200000                      20.7   

     MIN_Diversion_Rate_Total  SUM_Capture_Rate_Paper  AVG_Capture_Rate_Paper  \
0                        12.4                   580.7               41.478571   
1                        19.3                   120.2               40.066667   
2                        10.9                   552.6               39.471429   
3                        12.0                   491.2               40.933333   
4                        11.4                   473.2               39.433333   
..                        ...                     ...                     ...   
235                      14.1                   586.1               41.864286   
236                       9.7                   718.0               39.888889   
237                      10.1                   744.6               41.366667   
238                      11.6                   532.7               44.391667   
239                      19.5                   127.1               42.366667   

     MAX_Capture_Rate_Paper  MIN_Capture_Rate_Paper  SUM_Capture_Rate_MGP  \
0                      54.0                    30.3                1023.8   
1                      43.4                    36.0                 226.7   
2                      54.6                    28.1                 879.8   
3                      56.3                    30.7                 775.2   
4                      57.2                    29.0                 692.8   
..                      ...                     ...                   ...   
235                    61.6                    29.3                1044.6   
236                    55.0                    24.6                1080.4   
237                    56.0                    25.4                1052.4   
238                    58.7                    34.9                 726.3   
239                    45.9                    38.9                 247.5   

     AVG_Capture_Rate_MGP  MAX_Capture_Rate_MGP  MIN_Capture_Rate_MGP  \
0               73.128571                  92.5                  44.8   
1               75.566667                  77.7                  74.5   
2               62.842857                  81.3                  37.2   
3               64.600000                  84.6                  32.4   
4               57.733333                  71.8                  31.4   
..                    ...                   ...                   ...   
235             74.614286                 100.3                  47.2   
236             60.022222        

In [32]:
reciclagem_incendios= incendios.join(reciclagem, (incendios.case_year == reciclagem.Year) & (incendios.borough == reciclagem.Borough) & (incendios.month_name == reciclagem.Month_Name),"inner").drop(incendios.borough)

In [33]:
reciclagem_incendios=reciclagem_incendios.drop("month_name")

In [34]:
reciclagem_incendios=reciclagem_incendios.drop("case_year")

In [36]:
reciclagem_incendios = reciclagem_incendios.withColumn("month_name", 
   when((incendios.month == "1"), lit("January"))
     .when((incendios.month == "2"), lit("February")) 
     .when((incendios.month == "3"), lit("March")) 
     .when((incendios.month == "4"), lit("April")) 
     .when((incendios.month == "5"), lit("May")) 
     .when((incendios.month == "6"), lit("June")) 
     .when((incendios.month == "7"), lit("July")) 
     .when((incendios.month == "8"), lit("August")) 
     .when((incendios.month == "9"), lit("September")) 
     .when((incendios.month == "10"), lit("October"))   
     .when((incendios.month == "11"), lit("November"))  
     .when((incendios.month == "12"), lit("December")))

In [37]:
reciclagem_incendios.show()

+-----+----------+--------------------+----------+------------------+-------+-----------+---------+----------+------+---------------+-------+----------+-----+--------------+-------+-----------+----+-------------+------------------------+------------------------+------------------------+------------------------+----------------------+----------------------+----------------------+----------------------+--------------------+--------------------+--------------------+--------------------+----------------------+----------------------+----------------------+----------------------+----------+
|month|Appliances|Cooking_Carelessness|Electrical|Electrical_Devices|Heaters|Hot_Objects|Ignitable|Incendiary|Motors|Natural_Sources|No_Fire|Open_Flame|Other|Sem_Informacao|Smoking|Total_Fires|Year|      Borough|SUM_Diversion_Rate_Total|AVG_Diversion_Rate_Total|MAX_Diversion_Rate_Total|MIN_Diversion_Rate_Total|SUM_Capture_Rate_Paper|AVG_Capture_Rate_Paper|MAX_Capture_Rate_Paper|MIN_Capture_Rate_Paper|SUM_

In [38]:
spark.sql(
    """
    CREATE EXTERNAL TABLE PL7G1_GOLD.Incendios_Reciclagem (
        Year INT,
        Borough VARCHAR(50),
        month int,
        month_name string,
        SUM_Diversion_Rate_Total Double,
        AVG_Diversion_Rate_Total Double,
        MAX_Diversion_Rate_Total Double,
        MIN_Diversion_Rate_Total Double,
        SUM_Capture_Rate_Paper Double,
        AVG_Capture_Rate_Paper Double,
        MAX_Capture_Rate_Paper Double,
        MIN_Capture_Rate_Paper Double,
        SUM_Capture_Rate_MGP Double,
        AVG_Capture_Rate_MGP Double,
        MAX_Capture_Rate_MGP Double,
        MIN_Capture_Rate_MGP Double,
        SUM_Capture_Rate_Total Double,
        AVG_Capture_Rate_Total Double,
        MAX_Capture_Rate_Total Double,
        MIN_Capture_Rate_Total Double,        
        Appliances long,
        Cooking_Carelessness long,
        Electrical long, 
        Electrical_Devices long,
        Heaters long,
        Hot_Objects long,
        Ignitable long,
        Icendiary long,
        Motors long,
        Natural_Sources long,
        No_Fire long,
        Open_Flame long,
        Other long,
        Sem_Informacao long,
        Smoking long,
        Total_Fires long
    )
    USING DELTA
    LOCATION 'hdfs://hdfs-nn:9000/Poluicao_ar/Gold/PL7G1_GOLD.db/Incendios_Reciclagem'
    """
)

DataFrame[]

In [39]:
reciclagem_incendios \
    .write \
    .format("delta") \
    .mode("overwrite") \
    .option("mergeSchema", "true")\
    .save("hdfs://hdfs-nn:9000/Poluicao_ar/Gold/PL7G1_GOLD.db/Incendios_Reciclagem")

In [40]:
spark.table("PL7G1_GOLD.Incendios_Reciclagem").show()

+----+-------------+-----+----------+------------------------+------------------------+------------------------+------------------------+----------------------+----------------------+----------------------+----------------------+--------------------+--------------------+--------------------+--------------------+----------------------+----------------------+----------------------+----------------------+----------+--------------------+----------+------------------+-------+-----------+---------+---------+------+---------------+-------+----------+-----+--------------+-------+-----------+----------+
|Year|      Borough|month|month_name|SUM_Diversion_Rate_Total|AVG_Diversion_Rate_Total|MAX_Diversion_Rate_Total|MIN_Diversion_Rate_Total|SUM_Capture_Rate_Paper|AVG_Capture_Rate_Paper|MAX_Capture_Rate_Paper|MIN_Capture_Rate_Paper|SUM_Capture_Rate_MGP|AVG_Capture_Rate_MGP|MAX_Capture_Rate_MGP|MIN_Capture_Rate_MGP|SUM_Capture_Rate_Total|AVG_Capture_Rate_Total|MAX_Capture_Rate_Total|MIN_Capture_Rate

In [41]:
spark.sql("""
GENERATE symlink_format_manifest FOR TABLE delta.`hdfs://hdfs-nn:9000/Poluicao_ar/Gold/PL7G1_GOLD.db/Incendios_Reciclagem/`
""").show()

++
||
++
++



In [42]:
spark.sql("""
    CREATE EXTERNAL TABLE IF NOT EXISTS PL7G1_GOLD.Incendios_Reciclagem_Presto (
        Year INT,
        Borough VARCHAR(50),
        month int,
        month_name string,
        SUM_Diversion_Rate_Total Double,
        AVG_Diversion_Rate_Total Double,
        MAX_Diversion_Rate_Total Double,
        MIN_Diversion_Rate_Total Double,
        SUM_Capture_Rate_Paper Double,
        AVG_Capture_Rate_Paper Double,
        MAX_Capture_Rate_Paper Double,
        MIN_Capture_Rate_Paper Double,
        SUM_Capture_Rate_MGP Double,
        AVG_Capture_Rate_MGP Double,
        MAX_Capture_Rate_MGP Double,
        MIN_Capture_Rate_MGP Double,
        SUM_Capture_Rate_Total Double,
        AVG_Capture_Rate_Total Double,
        MAX_Capture_Rate_Total Double,
        MIN_Capture_Rate_Total Double,        
        Appliances long,
        Cooking_Carelessness long,
        Electrical long, 
        Electrical_Devices long,
        Heaters long,
        Hot_Objects long,
        Ignitable long,
        Icendiary long,
        Motors long,
        Natural_Sources long,
        No_Fire long,
        Open_Flame long,
        Other long,
        Sem_Informacao long,
        Smoking long,
        Total_Fires long
    )
    ROW FORMAT SERDE 'org.apache.hadoop.hive.ql.io.parquet.serde.ParquetHiveSerDe'
    STORED AS INPUTFORMAT 'org.apache.hadoop.hive.ql.io.SymlinkTextInputFormat'
    OUTPUTFORMAT 'org.apache.hadoop.hive.ql.io.HiveIgnoreKeyTextOutputFormat'
    LOCATION 'hdfs://hdfs-nn:9000/Poluicao_ar/Gold/PL7G1_GOLD.db/Incendios_Reciclagem/_symlink_format_manifest/'
    """).show()

++
||
++
++

